In [1]:
!curl -o Wonderland.txt "https://www.gutenberg.org/files/11/11-0.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  169k  100  169k    0     0  84680      0  0:00:02  0:00:02 --:--:-- 84680


# EIP Session 6 Assignment  (Phase-2 Session 2)


   1. Go through this Post: https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/ (Links to an external site.)
   
   2. Add these improvements to the final code described in the post:
    
        a. Predict 500 characters only
        
        b. Remove all the punctuation from the source text
        
        c. Train the model on padded sequences (Links to an external site.) rather than random sequences of characters. 
        
        d. Train the model for 100 epochs
        
        e. Add dropout to the input layer, remove it from the layer before dense layer. Use Dropout value of 0.1 everywhere
        
        f.Submit!


## The data is downloaded from the official website using curl.

## About dataset:

Download Link : https://www.gutenberg.org/ebooks/11

Alice's Adventures in Wonderland (commonly shortened to Alice in Wonderland) is an 1865 novel written by English author Charles Lutwidge Dodgson under the pseudonym Lewis Carroll. It tells of a young girl named Alice falling through a rabbit hole into a fantasy world populated by peculiar, anthropomorphic creatures.

We have used the plain text UTF-8 format.

In [2]:
!ls

sample_data  Wonderland.txt


# Text preprocessing:
  1.  Convert entire text to lower cases (To reduce the number of unuqie characters)
  2. Remove the unwanted new lines so that its a one single string where sentences are split by fullstops
  3. All punctuations apart from comma and fullstops are removed
  4. Individual characters are converted into integers based on a mapping
  5. Prepadding is done while generating sequential information in each sentences influenced by https://machinelearningmastery.com/develop-character-based-neural-language-model-keras/
  6. The target "y" is defined as the very first letter after the input "X" sequence in the sentence
 

In [0]:
# load ascii text and covert to lowercase
filename = "Wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [4]:
print(raw_text[:1000])

﻿project gutenberg’s alice’s adventures in wonderland, by lewis carroll

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  you may copy it, give it away or
re-use it under the terms of the project gutenberg license included
with this ebook or online at www.gutenberg.org


title: alice’s adventures in wonderland

author: lewis carroll

posting date: june 25, 2008 [ebook #11]
release date: march, 1994
last updated: october 6, 2016

language: english

character set encoding: utf-8

*** start of this project gutenberg ebook alice’s adventures in wonderland ***










alice’s adventures in wonderland

lewis carroll

the millennium fulcrum edition 3.0




chapter i. down the rabbit-hole

alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, ‘and what is the use of a book,’ tho

In [5]:
import numpy
import string
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.layers import Dropout
from keras.models import Model
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences


Using TensorFlow backend.


In [0]:
modified_text = ""
for i in range(0,len(raw_text)-1):
    if raw_text[i] == '\n':
        if raw_text[i+1] == '\n':
            continue
        elif raw_text[i-1] == '\n':
            modified_text += '. '
        else:
            modified_text += ' '
    else:
        modified_text += raw_text[i]

In [7]:
print(modified_text[:2000])

﻿project gutenberg’s alice’s adventures in wonderland, by lewis carroll. this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  you may copy it, give it away or re-use it under the terms of the project gutenberg license included with this ebook or online at www.gutenberg.org. title: alice’s adventures in wonderland. author: lewis carroll. posting date: june 25, 2008 [ebook #11] release date: march, 1994 last updated: october 6, 2016. language: english. character set encoding: utf-8. *** start of this project gutenberg ebook alice’s adventures in wonderland ***. alice’s adventures in wonderland. lewis carroll. the millennium fulcrum edition 3.0. chapter i. down the rabbit-hole. alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, ‘and what is the use of a book,’ thought alice ‘w

In [8]:
chars = sorted(list(set(modified_text)))
n_chars = len(modified_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
print("Characters: ",chars)

Total Characters:  163751
Total Vocab:  60
Characters:  [' ', '!', '#', '$', '%', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '“', '”', '\ufeff']


## Remove punctuation

In [9]:
exclude = set(string.punctuation)
print("Len: ",len(exclude))

Len:  32


In [10]:
exclude.remove('.')
#exclude.remove(';')
exclude.remove(',')
#exclude.remove("'")

print(exclude)
print("Len: ",len(exclude))

{'{', '\\', '@', '[', '"', '&', '!', '=', '~', '%', '$', '}', '_', '+', '/', ']', '*', '|', '#', '(', '-', ';', '<', '`', '^', '?', ':', ')', '>', "'"}
Len:  30


In [0]:
modified_text = ''.join(ch for ch in modified_text if ch not in exclude)

## Character to integer mapping definition

In [12]:
# create mapping of unique chars to integers
chars = sorted(list(set(modified_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
n_chars = len(modified_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
print("Characters: ",chars)

Total Characters:  161621
Total Vocab:  44
Characters:  [' ', ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '“', '”', '\ufeff']


## This is the method specified in the blog. It used randon set of continuous sequences

In [13]:
'''


'''

'\n\n# prepare the dataset of input to output pairs encoded as integers\nseq_length = 100\ndataX = []\ndataY = []\nfor i in range(0, n_chars - seq_length, 1):\n    seq_in = modified_text[i:i + seq_length]\n    seq_out = modified_text[i + seq_length]\n    dataX.append([char_to_int[char] for char in seq_in])\n    dataY.append(char_to_int[seq_out])\nn_patterns = len(dataX)\nprint("Total Patterns: ", n_patterns)\n\n'

 # Explanation of padding:
 
 Sentence: "You are beautiful"
 
 All patterns:
 X = "Y"            y = "o"
 
 X = "Yo"           y = "u"
 
 X = "You"          y = " "
 
 X = "You "         y = "a"
 
 and so on.... 
 And each X sentence is converted into character list, mapped to integers and pre padded to 100 characters with value = 0

In [14]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for sentence in modified_text.split('.'):
    for i in range(1,len(sentence)):
        if i <= seq_length:
            seq_in = sentence[:i]
        else:
            seq_in = sentence[i-seq_length:i]
        seq_out = modified_text[i]
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])
dataX = pad_sequences(dataX, maxlen=seq_length)
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  157901


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

# LSTM model definition in keras

## Adding drop out to the visible input layer of the first LSTM

Ref: https://keras.io/layers/recurrent/

The LSTM api from keras as 2 paramets namely dropout and recurrent_dropout.
1. dropout: Float between 0 and 1. Fraction of the units to drop for the linear transformation of the inputs.
2. recurrent_dropout: Float between 0 and 1. Fraction of the units to drop for the linear transformation of the recurrent state.

To achive the desired dropout at the input layer, the first parameter is used.

In [0]:
model_pro = Sequential()
model_pro.add(LSTM(256,dropout=0.1, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model_pro.add(Dropout(0.1))
model_pro.add(LSTM(256))
#model_pro.add(Dropout(0.1))
model_pro.add(Dense(y.shape[1], activation='softmax'))
model_pro.compile(loss='categorical_crossentropy', optimizer='adam')

In [22]:
model_pro.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_3 (Dense)              (None, 41)                10537     
Total params: 800,041
Trainable params: 800,041
Non-trainable params: 0
_________________________________________________________________


In [0]:
# define the checkpoint
filepath="TextGeneration.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [24]:
model_pro.fit(X, y, epochs=100, batch_size=256, callbacks=callbacks_list)

W0727 05:19:54.779409 139679689156480 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
157901/157901 [==============================] - 271s 2ms/step - loss: 2.9598

Epoch 00001: loss improved from inf to 2.95978, saving model to TextGeneration.hdf5
Epoch 2/100
157901/157901 [==============================] - 266s 2ms/step - loss: 2.9691

Epoch 00002: loss did not improve from 2.95978
Epoch 3/100
157901/157901 [==============================] - 265s 2ms/step - loss: 2.9005

Epoch 00003: loss improved from 2.95978 to 2.90054, saving model to TextGeneration.hdf5
Epoch 4/100
157901/157901 [==============================] - 263s 2ms/step - loss: 2.4742

Epoch 00004: loss improved from 2.90054 to 2.47420, saving model to TextGeneration.hdf5
Epoch 5/100
157901/157901 [==============================] - 262s 2ms/step - loss: 2.0570

Epoch 00005: loss improved from 2.47420 to 2.05703, saving model to TextGeneration.hdf5
Epoch 6/100
157901/157901 [==============================] - 262s 2ms/step - loss: 1.8540

Epoch 00006: loss improved from 2.05703 to 1.85398, saving 

## Saving the model

In [25]:
# Save the trained weights in to .h5 format
model_pro.save_weights("TextGeneraion_Weights_Best.h5")
print("Saved model to disk")


# Save the trained model in to .h5 format
model_pro.save("TextGeneration_Model_Best.h5")
print("Saved model to disk")

Saved model to disk
Saved model to disk


# Testing the model for 500 characters

In [28]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = list(dataX[start])
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print("\n")
end_res = ''
# generate characters
for i in range(500):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model_pro.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    end_res += result
    seq_in = [int_to_char[value] for value in pattern]
    #print(result, end = "")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print(end_res)
print("\n")
print("\nDone.")

Seed:
"  this time she found a little bottle on it, ‘which certainly was not here before,’ said alice, and r "


                                                                                                    project gutebberg’s alice’s adventures in wonderland, by lewis carroll. this bbook is for the see o                                          pr                                                        dddddddddllld       arreessssea e     d o   ellnd,, y  lewis carlll   t soo    ii  oo  th                                                                                            o   o   gutenberg’s 



Done.


# Using lower sequence length and verifing the effect

In [32]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 20
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = modified_text[i:i + seq_length]
    seq_out = modified_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)


Total Patterns:  161601


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
model_pro = Sequential()
model_pro.add(LSTM(256,dropout=0.1, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model_pro.add(Dropout(0.1))
model_pro.add(LSTM(256))
#model_pro.add(Dropout(0.1))
model_pro.add(Dense(y.shape[1], activation='softmax'))
model_pro.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="TextGeneration_1.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [35]:
model_pro.fit(X, y, epochs=100, batch_size=256, callbacks=callbacks_list)

Epoch 1/100
161601/161601 [==============================] - 67s 414us/step - loss: 2.8996

Epoch 00001: loss improved from inf to 2.89965, saving model to TextGeneration_1.hdf5
Epoch 2/100
161601/161601 [==============================] - 65s 400us/step - loss: 2.6599

Epoch 00002: loss improved from 2.89965 to 2.65987, saving model to TextGeneration_1.hdf5
Epoch 3/100
161601/161601 [==============================] - 65s 404us/step - loss: 2.4879

Epoch 00003: loss improved from 2.65987 to 2.48789, saving model to TextGeneration_1.hdf5
Epoch 4/100
161601/161601 [==============================] - 65s 405us/step - loss: 2.3231

Epoch 00004: loss improved from 2.48789 to 2.32307, saving model to TextGeneration_1.hdf5
Epoch 5/100
161601/161601 [==============================] - 65s 405us/step - loss: 2.1842

Epoch 00005: loss improved from 2.32307 to 2.18418, saving model to TextGeneration_1.hdf5
Epoch 6/100
161601/161601 [==============================] - 65s 404us/step - loss: 2.0810

Ep

In [36]:
# Save the trained weights in to .h5 format
model_pro.save_weights("TextGeneraion_1_Weights_Best.h5")
print("Saved model to disk")


# Save the trained model in to .h5 format
model_pro.save("TextGeneration_1_Model_Best.h5")
print("Saved model to disk")

Saved model to disk
Saved model to disk


In [37]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = list(dataX[start])
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print("\n")
end_res = ''
# generate characters
for i in range(500):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model_pro.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    end_res += result
    seq_in = [int_to_char[value] for value in pattern]
    #print(result, end = "")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print(end_res)
print("\n")
print("\nDone.")

Seed:
" phasis, looking hard "


 in c hare wroembi, rnthiz yhem ier hergr  andrhe pa kovt it  anoca dow mnterflat.’ert onte alice, andrie tetlap. su sle wetts ofcrt thap.’or’t  itencutallne       i’ tme,eisce ma’s tr onte’id cnxergsmed tuadtisgipe foassurinati. hirgat inpirakd.’hy, tle felling outenf, yhir tuove whemswle tes,tn the had a vhg ootk abok sp the toat chgs in the’nnnt  and uatie  and soly toontiyed tudetingsta cre fonleted tuclte su oes.oo tif eeien allce ln a hueate inplan inmoth, andr rf tuuject gute bnice’s ard 



Done.


# Conclutions:

## 1. When the punctuation like " ' " and " " " and still retained in the data, the model was able to predict direct speech sentences as well. SO work can be done on this as well

## 2. Character level text generation will require larger network and more epochs to learn well. So the idea of iteration through the sentence was helpful and verified

## 3. Pre padding sequences actually didn't help as it gives too many unwanted white spaces in its outout

## 4. When using smaller sequences, say 20 instead of 100 (as done in the second case), the output text predected in that proper both semantically and grammatically
